In [1]:
import json, requests, jdc
import pandas as pd
import numpy as np

Create account from [API](https://www.ura.gov.sg/maps/api/#introduction) link

In [2]:
# this class will be defined across multiple cells
class call_api():
    def __init__(self, access_key=None):
        # variables
        self.access_key = None
        self.token = None
        self.header = None
        self.residential_property_transactions = None
        self.transactions = None
        
        # access key from account creation
        self.access_key = '77144a24-fc1b-4358-b5b7-45d3a94a7aec' if not access_key else access_key

        # generate token from access key valid for one day
        url = 'https://www.ura.gov.sg/uraDataService/insertNewToken.action'
        header = {'AccessKey': f'{access_key}'}
        acc_info = requests.get(url, headers = header).json()
        assert acc_info['Status'] == 'Success', 'Too many tokens have been generated recently. Use another access_key'
        self.token = acc_info['Result']
        self.header = {'AccessKey': f'{self.access_key}', 'Token': self.token}

In [3]:
# instantiate for later use
api = call_api('89d0c48b-dcb2-4798-89f5-c2f4f5caddd8')

## Residential property transactions


This data service will return past 5 years of private residential property transaction records in JSON format. As transaction records > 5 years ago could be modified/aborted, we would advise to refresh your database on a daily basis and just retain the latest 5 years record for better accuracy.

Update Frequency: End of day of every Tuesday and Friday

- `project`	The name of the project
- `street`	The street name that the project is on.
- `marketSegment`	The market segment that the property falls in.
    - `CCR` – Core Central Region
    - `RCR` – Rest of Central Region
    - `OCR` – Outside Central Region
- `x`	The x coordinates of the address of the property in SVY21 format. Important: This is the location of the property and does not represent the location of the transacted unit.
- `y`	The y coordinates of the address of the property in SVY21 format. Important: This is the location of the property and does not represent the location of the transacted unit.
- `transaction`	An array of transactions for this property
    - `propertyType`	The property type of the transacted property. Note that there are properties with a mixture of property types.
        - `Strata` Detached
        - `Strata` Semidetached
        - `Strata` Terrace
        - `Detached`
        - `Semi`-detached
        - `Terrace`
        - `Apartment`
        - `Condominium`
        - `Executive` Condominium
    - `district`	The postal district that the transacted property falls in. Note that there are properties that fall across multiple postal district.
    - `tenure`	The tenure of the transacted property. Note that there are properties that have units with multiple tenures.
        - `Freehold`
        - `xx` yrs lease commencing from yyyy
    - `typeOfSale`	The type of sale
        - `1` – New Sale
        - `2` – Sub Sale
        - `3` – Resale
    - `noOfUnits`	The number of units in this transaction. The value for New Sale will always be 1. The value for Resale or Sub Sale could be greater than 1 depending on the number of units lodged for the caveat.
    - `price`	The transacted price nettPrice
    - `nettPrice`	The nett transacted price, excluding discounts if any. This field is only applicable for New Sale where discounts were given.
    - `area`	The land/floor area of the transacted unit in square metre.
    - `typeOfArea`	The type of area of the transacted unit.
        - `Strata`
        - `Land`
        - `Unknown`
    - `floorRange`	The floor range that the transacted unit falls within.
        - `-`
        - `B1`-B5
        - `B6`-B10
        - `01`-05
        - `06`-10
        ...
    - `contractDate`	The data of sale for New Sale records or option exercised date for Resale and Sub Sale records. Field is in format of mmyy e.g. 1215 represents Dec 2015.

In [9]:
%%add_to call_api
# above allows class to be defined across multiple cells
def call_residential_property_transactions(self, call=False):
    # call api once and keep in memory
    if call:
        url = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=1'
        data = requests.get(url, headers = self.header).json()
    else:
        assert self.residential_property_transactions != None, 'call_residential_property_transactions() has already been called once'
        data = self.residential_property_transactions
    self.residential_property_transactions = data['Result']

In [10]:
# call once
api.call_residential_property_transactions(call=True)

In [11]:
# peek at the data
api.residential_property_transactions[:2]

[{'street': 'ZEHNDER ROAD',
  'project': 'LANDED HOUSING DEVELOPMENT',
  'transaction': [{'area': '314',
    'floorRange': '-',
    'noOfUnits': '1',
    'contractDate': '0618',
    'typeOfSale': '3',
    'price': '4750000',
    'propertyType': 'Semi-detached',
    'district': '05',
    'typeOfArea': 'Land',
    'tenure': 'Freehold'},
   {'area': '524.3',
    'floorRange': '-',
    'noOfUnits': '1',
    'contractDate': '0220',
    'typeOfSale': '3',
    'price': '5500000',
    'propertyType': 'Semi-detached',
    'district': '05',
    'typeOfArea': 'Land',
    'tenure': 'Freehold'},
   {'area': '308',
    'floorRange': '-',
    'noOfUnits': '1',
    'contractDate': '0918',
    'typeOfSale': '3',
    'price': '5000000',
    'propertyType': 'Semi-detached',
    'district': '05',
    'typeOfArea': 'Land',
    'tenure': 'Freehold'}],
  'marketSegment': 'RCR'},
 {'street': 'NEO PEE TECK LANE',
  'project': 'LANDED HOUSING DEVELOPMENT',
  'transaction': [{'area': '159.3',
    'floorRange': '

In [12]:
%%add_to call_api
def by_region(self, region):
    '''
    returns list of dictionaries containing information of each house
    eg: [{house1 details},
         {house2 details}, ...
        ]
    '''
    assert self.residential_property_transactions != None, 'call residential_property_transactions() first'
    assert region in ['CCR', 'RCR', 'OCR'], 'region must be in ["CCR", "RCR", "OCR"]'
    self.transactions = None

    for street in self.residential_property_transactions:
        if street['marketSegment'] == region:
            if self.transactions != None:
                self.transactions += street['transaction']
            else:
                self.transactions = street['transaction']

In [13]:
# filter by CCR
api.by_region('CCR')
# peek first 2 house info
api.transactions[:2]

[{'area': '258',
  'floorRange': '01-05',
  'noOfUnits': '1',
  'contractDate': '0616',
  'typeOfSale': '3',
  'price': '3800000',
  'propertyType': 'Condominium',
  'district': '04',
  'typeOfArea': 'Strata',
  'tenure': '99 yrs lease commencing from 2007'},
 {'area': '226',
  'floorRange': '01-05',
  'noOfUnits': '1',
  'contractDate': '0920',
  'typeOfSale': '3',
  'price': '3400000',
  'propertyType': 'Condominium',
  'district': '04',
  'typeOfArea': 'Strata',
  'tenure': '99 yrs lease commencing from 2007'}]

Now we need think how to plot the graph first, we wan just all the price plot or like can toggle by type of house